In [5]:
from mermaid import Mermaid
from pydantic import BaseModel

## What are we trying to do?

In [6]:
Mermaid("flowchart LR\nA[LLM Invocations Synthetic/Human Inputs] --> B[Unit Tests]\nA --> C[Logging Traces]\nB & C --> D[Eval & Curation 1. Human review 2. Model-based 3. A/B tests]\nD -->|Automate this more over time| E[Fine Tune/ Curated Data]\nD --> F[Prompt Eng]\nE & F -->|Versioned Together| G[Improve Model]\nG --> A")

Let's assume there exists python library inspired by DSPy. The ideals of DSPy are:
1. Modular LLM Programs
2. Scoring/evaluation
3. Optimization

## What is a prompt?
1. Prompt Instruction -> "Based on the context provided, answer the question" #BayesianSignatureOpt does in DSPy
2. Few Shot Examples: Selection, Metadata, Re-ordering -> RAG Evaluation -> ragas #Compilation, labeled demos -> MIPRO
3. Reference information to answer the current question -> ragas # Retriever 
4. Teaching the LLM to use the Examples well. Synthetic Data e.g. reasoning about the question and answer -> Synthetic Data Generation

Option 1: `question, context -> answer`
Option 2: `question, context, intermediate_reasoning, reasoning -> final_answer` ✅


"""
pool_of_demos -> selected_demos
?? (missing inputs) -> labeled_demos 
labeled_demos, selected_demos ->few_shot_examples 
retrieval/search -> reference
few_shot_examples, reference -> context
question, context -> intermediate_reasoning
question, context, intermediate_reasoning -> reasoning 
question, context, reasoning -> final_answer
"""

## What are the main components of this library?
1. Some way to capture relationships between inputs and outputs. If there is an intermediate output, we should optimize for that too ✅  
2. Some way to "score" the output -> Loss Function (composition of functions) ✅ -> Score using faithfulness from ragas (1-Hallucination)
3. Some way to use this score, to update the steps in between -> Backpropagation / Hyperparameter Optimization (hyperopt) "Update"

In [ ]:
from pydantic import BaseModel, Field
from typing import Union, List


class Question(BaseModel):
    name: str = "question"
    value: str = Field(..., description="The question to be answered")

class Context(BaseModel):
    name: str = "context"
    value: str = Field(..., description="The context to be used to answer the question")

class Response(BaseModel):
    name: str = "response"
    value: str = Field(..., description="The response to the question")

class StringAnswer(Response):
    name: str = "answer"
    value: str = Field(..., description="The answer to the question")

class NumberAnswer(Response):
    name: str = "answer"
    value: Union[float, int] = Field(..., description="The answer to the question")

class ListAnswer(Response):
    name: str = "answer"
    value: List[str] = Field(..., description="The answer to the question")

class IntermediateReasoning(BaseModel):
    name: str = "intermediate_reasoning"
    value: str = Field(..., description="The intermediate reasoning to be used to answer the question")

class Reasoning(BaseModel):
    name: str = "reasoning"
    value: str = Field(..., description="The reasoning to be used to answer the question")

"""Scenario 1: question, context -> answer""" # function A
def question_context_to_answer(**kwargs) -> StringAnswer:
    """
    This function takes a question and a context, and returns an answer.
    If the system does return a list, it will now be read as a string or expected to be a string. 
    """
    question, context = kwargs['question'], kwargs['context']
    return StringAnswer(value=f"The answer to the question {question}, {context}")

"""Scenario 2: question, context, reasoning -> final_answer""" # Function B i.e. B(A)
def question_context_with_intermediate_reasoning(**kwargs) -> StringAnswer:
    """
    This function takes a question and a context, and returns an answer.
    If the system does return a list, it will now be read as a string or expected to be a string. 
    """
    reasoning = Reasoning(value=f"The reasoning to be used to answer the question: {kwargs['question']}, based on {kwargs['context']}")
    context = Context(value=f"The context to be used to answer the question: {kwargs['context']} and uses the reasoning: {reasoning.value}")
    return question_context_to_answer(question=kwargs["question"], context=context)

"""Scenario 3: question, context, intermediate_reasoning, reasoning -> final_answer""" # Function C
def question_context_w_reasoning(question: Question, context: Context) -> StringAnswer:
    """
    This function takes a question and a context, and returns an answer.
    If the system does return a list, it will now be read as a string or expected to be a string. 
    """
    intermediate_reasoning = Reasoning(...)
    reasoning = question_context_with_intermediate_reasoning(...)
    updated_context = context + reasoning
    return question_context_to_answer(question, updated_context)

"""Scenario 3: 
question, context -> intermediate_reasoning
question, context, intermediate_reasoning -> reasoning 
question, context, reasoning -> final_answer"""



## Directed Acyclic Graph

# Copy what PyTorch got right

In [ ]:
import dspy
"""Scenario 2: question, context, reasoning -> final_answer""" # Function B i.e. B(A)
class GenerateAnswer(dspy.Signature):
    """Answer questions based on the context."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    reasoning = dspy.InputField()
    answer = dspy.OutputField()

class Reasoning(dspy.Signature):
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    reasoning = dspy.OutputField()

class RAG(dspy.Module):
    def __init__(self, collection_name="rag_contexts", num_passages=10):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages) # function
        self.generate_reasoning = dspy.Predict(Reasoning)
        self.generate_answer = dspy.Predict(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages
        reason = self.generate_reasoning(question=question, context=context)
        prediction = self.generate_answer(context=context, question=question, reasoning=reason)
        return dspy.Prediction(answer=prediction.answer)

In [4]:


# from nkpy.compilers import MIPRO
# from nkpy.evaluate.evaluate import Evaluate
# from metrics.gold_passages_retrieved import gold_passages_retrieved
# from models.simplified_baleen import SimplifiedBaleen
# from metrics.validate_context_and_answer_and_hops import validate_context_and_answer_and_hops

# # Assume devset, trainset, uncompiled_baleen, and compiled_baleen are defined

# # Initialize the evaluate function
# evaluate_on_hotpotqa = Evaluate(
#     devset=devset,
#     num_threads=1,
#     display_progress=True,
#     display_table=5
# )

# # Evaluate the uncompiled Baleen model
# uncompiled_baleen_retrieval_score = evaluate_on_hotpotqa(
#     uncompiled_baleen,
#     metric=gold_passages_retrieved,
#     display=False
# )

# # Evaluate the compiled Baleen model
# compiled_baleen_retrieval_score = evaluate_on_hotpotqa(
#     compiled_baleen,
#     metric=gold_passages_retrieved
# )

# print(f"## Retrieval Score for uncompiled Baleen: {uncompiled_baleen_retrieval_score}")
# print(f"## Retrieval Score for compiled Baleen: {compiled_baleen_retrieval_score}")

# # Compile the model using MIPRO
# compiler = MIPRO(metric=validate_context_and_answer_and_hops)
# compiled_baleen = compiler.compile(
#     SimplifiedBaleen(),
#     teacher=SimplifiedBaleen(passages_per_hop=2),
#     trainset=trainset
# )

ModuleNotFoundError: No module named 'nkpy'

## MIPRO Overall

In [9]:
Mermaid("graph LR\nA[Start] --> B[Initialize]\nB --> C[Propose]\nC --> D[Update]\nD --> E{Extract Optimized Sets?}\nE -->|Yes| F[Evaluate Best Candidates]\nE -->|No| C\nF --> G{All Trials Completed?}\nG -->|Yes| H[Return Optimal Assignment]\nG -->|No| C")

## MIPRO "Propose"

In [8]:
Mermaid("graph TD\nA[Start Propose] --> B[Prepare LLM Inputs]\nB --> C[LLM Generates Candidate Instructions]\nC --> D[Process & Rank LLM Output]\nD --> E[TPE Samples Instructions]\nE --> F[TPE Samples Demos]\nF --> G[Create Partial Assignments]\nG --> H[End Propose]")